In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder
from sklearn.model_selection import KFold, cross_val_score
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from pycaret.regression import setup, compare_models
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import numpy as np
import pandas as pd
import multiprocessing

In [40]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import BayesianRidge, OrthogonalMatchingPursuit, LinearRegression, Lasso, Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import catboost

# Read train and test sets

In [3]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

# Store both dfs lengths

In [4]:
m_train = df_train.shape[0]
m_test = df_test.shape[0]
m_train, m_test

(1460, 1459)

# Concat the dfs

In [5]:
df = pd.concat([df_train, df_test])
assert df.shape[0] == m_train + m_test

In [6]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


# Save target

In [7]:
target_col, target = "SalePrice", df["SalePrice"]

#  Drop Id and target columns

In [8]:
cols_to_drop = ["Id", target_col]
df.drop(cols_to_drop, axis=1, inplace=True)

# View and fix columns data-types

In [9]:
df.select_dtypes(object).columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [10]:
df.select_dtypes(np.number).columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')

In [11]:
num_to_obj_cols = ['MSSubClass', 'MoSold']
df[num_to_obj_cols] = df[num_to_obj_cols].astype(object)

In [12]:
cols_cat = df.select_dtypes(object).columns
cols_num = df.select_dtypes(np.number).columns

# Impute categorical columns

In [13]:
cols_cat_na = df[cols_cat].isnull().sum()[df[cols_cat].isnull().sum() > 0]
cols_cat_na

MSZoning           4
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinType2      80
Electrical         1
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageFinish     159
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
dtype: int64

In [14]:
mode_filled_cols = ["MSZoning", "Utilities", "Exterior1st", "Exterior2nd", "MasVnrType", "Electrical", "KitchenQual", "Functional", "SaleType"]
for col in mode_filled_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

none_filled_cols = ["Alley", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "FireplaceQu", "GarageType", "GarageFinish", "GarageQual", "GarageCond", "PoolQC", "Fence", "MiscFeature"]
for col in none_filled_cols:
    df[col].fillna("None", inplace=True)
    
df[cols_cat].isnull().sum().sum()

0

# Impute numerical columns

In [15]:
cols_num_na = df[cols_num].isnull().sum()[df[cols_num].isnull().sum() > 0]
cols_num_na

LotFrontage     486
MasVnrArea       23
BsmtFinSF1        1
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
GarageYrBlt     159
GarageCars        1
GarageArea        1
dtype: int64

In [16]:
zero_filled_cols = ["BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "BsmtFullBath", "BsmtHalfBath"]
for col in zero_filled_cols:
    df[col].fillna(0, inplace=True)

In [17]:
cols_num_na = df[cols_num].isnull().sum()[df[cols_num].isnull().sum() > 0]
cols_num_na

LotFrontage    486
MasVnrArea      23
GarageYrBlt    159
GarageCars       1
GarageArea       1
dtype: int64

In [18]:
def gbr_imputer(df, cols_num_na):
    imputer = KNNImputer(n_neighbors=8)
    df_c = df.copy()
    df_d = pd.get_dummies(df_c)
    cat_cols = df_d.select_dtypes(object).columns.to_list()
    for col in tqdm(cols_num_na):
        df_i = df_d.copy()
        X, y = df_i.drop(col, axis=1), df[col]
        X[:] = imputer.fit_transform(X)
        
        train_indexes, test_indexes = ~df_i[col].isnull(), df_i[col].isnull()
        X_train, X_test = X.loc[train_indexes, :], X.loc[test_indexes, :]
        y_train, y_test = y.loc[train_indexes], y.loc[test_indexes]
        
        model = CatBoostRegressor(max_depth=8, random_seed=10,
                                  subsample=0.65, n_estimators=1000,
                                  cat_features=cat_cols)
        model.fit(X_train, y_train)
        df.loc[df[col].isnull(), col] = model.predict(X_test)
    return df

In [19]:
df = gbr_imputer(df, cols_num_na.index)
df.isnull().sum().sum()

  0%|                                                     | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.047118
0:	learn: 22.8651061	total: 69.4ms	remaining: 1m 9s
1:	learn: 22.4009335	total: 95.7ms	remaining: 47.8s
2:	learn: 21.9445217	total: 123ms	remaining: 40.7s
3:	learn: 21.5450188	total: 133ms	remaining: 33.1s
4:	learn: 21.1275746	total: 148ms	remaining: 29.4s
5:	learn: 20.7285771	total: 156ms	remaining: 25.9s
6:	learn: 20.3540435	total: 172ms	remaining: 24.5s
7:	learn: 19.9960252	total: 182ms	remaining: 22.6s
8:	learn: 19.6384461	total: 193ms	remaining: 21.3s
9:	learn: 19.3433983	total: 210ms	remaining: 20.8s
10:	learn: 19.0360964	total: 222ms	remaining: 20s
11:	learn: 18.7936968	total: 235ms	remaining: 19.4s
12:	learn: 18.5446860	total: 258ms	remaining: 19.6s
13:	learn: 18.2640645	total: 281ms	remaining: 19.8s
14:	learn: 18.0282059	total: 295ms	remaining: 19.4s
15:	learn: 17.7900793	total: 306ms	remaining: 18.8s
16:	learn: 17.5428123	total: 362ms	remaining: 20.9s
17:	learn: 17.3214342	total: 388ms	remaining: 21.2s
18:	learn: 17.1690972	total: 395ms	remaining

161:	learn: 10.5192531	total: 2.58s	remaining: 13.4s
162:	learn: 10.4908491	total: 2.59s	remaining: 13.3s
163:	learn: 10.4706890	total: 2.61s	remaining: 13.3s
164:	learn: 10.4495234	total: 2.63s	remaining: 13.3s
165:	learn: 10.4175960	total: 2.65s	remaining: 13.3s
166:	learn: 10.4044389	total: 2.65s	remaining: 13.2s
167:	learn: 10.3891284	total: 2.66s	remaining: 13.2s
168:	learn: 10.3560379	total: 2.67s	remaining: 13.1s
169:	learn: 10.3235662	total: 2.67s	remaining: 13.1s
170:	learn: 10.3085671	total: 2.68s	remaining: 13s
171:	learn: 10.2893014	total: 2.69s	remaining: 13s
172:	learn: 10.2798679	total: 2.71s	remaining: 12.9s
173:	learn: 10.2471812	total: 2.72s	remaining: 12.9s
174:	learn: 10.2348019	total: 2.74s	remaining: 12.9s
175:	learn: 10.2288971	total: 2.75s	remaining: 12.9s
176:	learn: 10.1965296	total: 2.76s	remaining: 12.8s
177:	learn: 10.1750179	total: 2.77s	remaining: 12.8s
178:	learn: 10.1633518	total: 2.78s	remaining: 12.8s
179:	learn: 10.1514884	total: 2.81s	remaining: 12.

327:	learn: 7.8433565	total: 5.11s	remaining: 10.5s
328:	learn: 7.8299051	total: 5.12s	remaining: 10.4s
329:	learn: 7.8278547	total: 5.14s	remaining: 10.4s
330:	learn: 7.8172502	total: 5.15s	remaining: 10.4s
331:	learn: 7.8093168	total: 5.16s	remaining: 10.4s
332:	learn: 7.7941597	total: 5.17s	remaining: 10.3s
333:	learn: 7.7797599	total: 5.18s	remaining: 10.3s
334:	learn: 7.7672140	total: 5.19s	remaining: 10.3s
335:	learn: 7.7512479	total: 5.21s	remaining: 10.3s
336:	learn: 7.7400578	total: 5.21s	remaining: 10.3s
337:	learn: 7.7289989	total: 5.24s	remaining: 10.3s
338:	learn: 7.7119396	total: 5.27s	remaining: 10.3s
339:	learn: 7.6993414	total: 5.3s	remaining: 10.3s
340:	learn: 7.6941205	total: 5.31s	remaining: 10.3s
341:	learn: 7.6786417	total: 5.32s	remaining: 10.2s
342:	learn: 7.6635078	total: 5.34s	remaining: 10.2s
343:	learn: 7.6497882	total: 5.35s	remaining: 10.2s
344:	learn: 7.6342611	total: 5.36s	remaining: 10.2s
345:	learn: 7.6328753	total: 5.37s	remaining: 10.2s
346:	learn: 7

491:	learn: 6.2631892	total: 7.59s	remaining: 7.84s
492:	learn: 6.2575450	total: 7.61s	remaining: 7.82s
493:	learn: 6.2514335	total: 7.62s	remaining: 7.8s
494:	learn: 6.2428858	total: 7.62s	remaining: 7.78s
495:	learn: 6.2374989	total: 7.63s	remaining: 7.75s
496:	learn: 6.2369411	total: 7.63s	remaining: 7.73s
497:	learn: 6.2303359	total: 7.64s	remaining: 7.7s
498:	learn: 6.2213633	total: 7.65s	remaining: 7.68s
499:	learn: 6.2204213	total: 7.66s	remaining: 7.66s
500:	learn: 6.2074346	total: 7.68s	remaining: 7.65s
501:	learn: 6.1939397	total: 7.69s	remaining: 7.63s
502:	learn: 6.1852924	total: 7.72s	remaining: 7.63s
503:	learn: 6.1795564	total: 7.72s	remaining: 7.6s
504:	learn: 6.1698630	total: 7.73s	remaining: 7.58s
505:	learn: 6.1635055	total: 7.74s	remaining: 7.55s
506:	learn: 6.1547986	total: 7.74s	remaining: 7.53s
507:	learn: 6.1376110	total: 7.75s	remaining: 7.51s
508:	learn: 6.1307642	total: 7.76s	remaining: 7.48s
509:	learn: 6.1236219	total: 7.77s	remaining: 7.46s
510:	learn: 6.1

...

663:	learn: 38.7924206	total: 9.92s	remaining: 5.02s
664:	learn: 38.7631204	total: 9.93s	remaining: 5s
665:	learn: 38.7020878	total: 9.96s	remaining: 5s
666:	learn: 38.6552405	total: 9.98s	remaining: 4.99s
667:	learn: 38.6516570	total: 10s	remaining: 4.97s
668:	learn: 38.6204940	total: 10s	remaining: 4.96s
669:	learn: 38.6013270	total: 10s	remaining: 4.94s
670:	learn: 38.5850257	total: 10s	remaining: 4.92s
671:	learn: 38.5363692	total: 10s	remaining: 4.9s
672:	learn: 38.4844097	total: 10s	remaining: 4.88s
673:	learn: 38.4356989	total: 10s	remaining: 4.86s
674:	learn: 38.3861975	total: 10.1s	remaining: 4.84s
675:	learn: 38.3794617	total: 10.1s	remaining: 4.82s
676:	learn: 38.3241362	total: 10.1s	remaining: 4.8s
677:	learn: 38.2656739	total: 10.1s	remaining: 4.78s
678:	learn: 38.2061761	total: 10.1s	remaining: 4.76s
679:	learn: 38.1367617	total: 10.1s	remaining: 4.75s
680:	learn: 38.0985166	total: 10.1s	remaining: 4.73s
681:	learn: 38.0814879	total: 10.1s	remaining: 4.71s
682:	learn: 38.

827:	learn: 33.5227210	total: 12.2s	remaining: 2.54s
828:	learn: 33.5066633	total: 12.2s	remaining: 2.52s
829:	learn: 33.4896379	total: 12.2s	remaining: 2.5s
830:	learn: 33.4861152	total: 12.2s	remaining: 2.49s
831:	learn: 33.4277378	total: 12.3s	remaining: 2.47s
832:	learn: 33.3953719	total: 12.3s	remaining: 2.46s
833:	learn: 33.3903420	total: 12.3s	remaining: 2.44s
834:	learn: 33.3381016	total: 12.3s	remaining: 2.43s
835:	learn: 33.3330886	total: 12.3s	remaining: 2.41s
836:	learn: 33.2715623	total: 12.3s	remaining: 2.4s
837:	learn: 33.2185669	total: 12.3s	remaining: 2.38s
838:	learn: 33.2159295	total: 12.3s	remaining: 2.37s
839:	learn: 33.1903849	total: 12.3s	remaining: 2.35s
840:	learn: 33.1265549	total: 12.4s	remaining: 2.33s
841:	learn: 33.0946015	total: 12.4s	remaining: 2.32s
842:	learn: 33.0563252	total: 12.4s	remaining: 2.31s
843:	learn: 33.0536631	total: 12.4s	remaining: 2.29s
844:	learn: 33.0152697	total: 12.4s	remaining: 2.28s
845:	learn: 32.9783994	total: 12.4s	remaining: 2

985:	learn: 29.2155055	total: 14.5s	remaining: 206ms
986:	learn: 29.1986676	total: 14.5s	remaining: 191ms
987:	learn: 29.1903139	total: 14.5s	remaining: 176ms
988:	learn: 29.1636613	total: 14.5s	remaining: 162ms
989:	learn: 29.1473860	total: 14.5s	remaining: 147ms
990:	learn: 29.1063222	total: 14.5s	remaining: 132ms
991:	learn: 29.0694489	total: 14.5s	remaining: 117ms
992:	learn: 29.0295002	total: 14.6s	remaining: 103ms
993:	learn: 29.0023269	total: 14.6s	remaining: 87.9ms
994:	learn: 28.9739587	total: 14.6s	remaining: 73.3ms
995:	learn: 28.9443718	total: 14.6s	remaining: 58.7ms
996:	learn: 28.9088407	total: 14.6s	remaining: 44ms
997:	learn: 28.9010819	total: 14.6s	remaining: 29.3ms
998:	learn: 28.8835614	total: 14.7s	remaining: 14.7ms
999:	learn: 28.8585479	total: 14.7s	remaining: 0us


100%|█████████████████████████████████████████████| 5/5 [01:29<00:00, 17.85s/it]


0

# Feature selection

In [20]:
def get_del_cols(df):
    # return the columns where no more than one value in the categorical features
    # exists in the test data - those features can be ignored
    cols_to_drop = []
    for col in df.select_dtypes(object).columns:
        col_vals = df[col].unique()
        n_vals = len(col_vals)
        n_irrelevant = 0
        for val in col_vals:
            if val not in df[col][m_train:].values:
                n_irrelevant += 1
        if n_irrelevant >= n_vals - 1:
            cols_to_drop.append(col)
    return cols_to_drop

In [21]:
cols_to_drop = get_del_cols(df)
df.drop(cols_to_drop, axis=1, inplace=True)
cols_to_drop

['Utilities']

# Change ordinal columns to numeric, and encode accordingly

In [22]:
df.select_dtypes(object).columns

Index(['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'MoSold',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [23]:
oe = OrdinalEncoder(categories=[['Reg', 'IR1', 'IR2', 'IR3']])
df.loc[:, "LotShape"] = oe.fit_transform(df[["LotShape"]])
df["LotShape_sq"] = df["LotShape"] ** 2

In [24]:
oe = OrdinalEncoder(categories=[['Gtl', 'Mod', 'Sev']])
df.loc[:, "LandSlope"] = oe.fit_transform(df[["LandSlope"]])
df["LandSlope_sq"] = df["LandSlope"] ** 2

In [25]:
qual_oe = OrdinalEncoder(categories=[['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex']])
for col in ["ExterQual", "ExterCond", 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']:
    df.loc[:, col] = qual_oe.fit_transform(df[[col]])
    df[col + "sq"] = df[col] ** 2

In [26]:
oe = OrdinalEncoder(categories=[['None', 'No', 'Mn', 'Av', 'Gd']])
df.loc[:, "BsmtExposure"] = oe.fit_transform(df[["BsmtExposure"]])
df["BsmtExposure_sq"] = df["BsmtExposure"] ** 2

In [27]:
oe = OrdinalEncoder(categories=[['None', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']])
for col in ["BsmtFinType1", "BsmtFinType2"]:
    df.loc[:, col] = oe.fit_transform(df[[col]])
    df[col + "sq"] = df[col] ** 2

In [28]:
oe = OrdinalEncoder(categories=[['None', 'Unf', 'RFn', 'Fin']])
df.loc[:, "GarageFinish"] = oe.fit_transform(df[["GarageFinish"]])
df["GarageFinishsq"] = df["GarageFinish"] ** 2

In [29]:
oe = OrdinalEncoder(categories=[['N', 'P', 'Y']])
df.loc[:, "PavedDrive"] = oe.fit_transform(df[["PavedDrive"]])
df["PavedDrivesq"] = df["PavedDrive"] ** 2

In [30]:
oe = OrdinalEncoder(categories=[['None', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']])
df.loc[:, "Fence"] = oe.fit_transform(df[["Fence"]])
df["Fencesq"] = df["Fence"] ** 2

# Feature Engineering

In [31]:
n_stories_dict = {
    "1Story": 1.0,
    "1.5Fin": 1.5,
    "1.5Unf": 1.5,
    "2Story": 2.0,
    "2.5Fin": 2.5,
    "2.5Unf": 2.5,
    "SFoyer": 2.0,
    "SLvl": 2.0,
}

df["n_stories"] = df["HouseStyle"].replace(n_stories_dict)
df["n_stories"].value_counts()

1.0    1471
2.0    1083
1.5     333
2.5      32
Name: n_stories, dtype: int64

In [32]:
df["age_sold"] = df["YrSold"] - df["YearBuilt"]
df["age_sold_Remod"] = df["YrSold"] - df["YearRemodAdd"]
df["GarageYrSold"] = df["YrSold"] - df["GarageYrBlt"]

In [33]:
df["CentralAir"] = df["CentralAir"].replace({"Y": 1, "N": 0})
df["CentralAir"].value_counts()

1    2723
0     196
Name: CentralAir, dtype: int64

In [34]:
df.select_dtypes(np.number).columns

Index(['LotFrontage', 'LotArea', 'LotShape', 'LandSlope', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'HeatingQC', 'CentralAir', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Fireplaces', 'FireplaceQu', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscVal', 'YrSold',
       'LotShape_sq', 'LandSlope_sq', 'ExterQualsq', 'ExterCondsq',
       'BsmtQualsq', 'BsmtCondsq', 'HeatingQCsq', 'KitchenQualsq',
       'FireplaceQusq', 'GarageQualsq', 'GarageCondsq',

In [35]:
df["total_area"] = df['LotArea'] + df['TotalBsmtSF'] + df['GrLivArea'] + df['1stFlrSF'] + df['2ndFlrSF'] + df['GarageArea']

df['total_area_hq'] = df["total_area"] - df['LowQualFinSF']

df["SqFtPerRoom"] = df["GrLivArea"] / (df["TotRmsAbvGrd"] +
                                       df["FullBath"] +
                                       df["HalfBath"] +
                                       df["KitchenAbvGr"])

df['Total_Home_Quality'] = df['OverallQual'] * df['OverallCond']
df['Total_bsmt_Quality'] = df['BsmtQual'] * df['BsmtCond']
df['Total_garage_Quality'] = df['GarageQual'] * df['GarageCond']

df["weighted_home_area"] = df["total_area"] * df['Total_Home_Quality'] 
df["weighted_bsmt_area"] = df["TotalBsmtSF"] * df['Total_bsmt_Quality'] 
df["weighted_garage_area"] = df["GarageArea"] * df['Total_garage_Quality']



df["area_qu"] = df["total_area"] 

# Split test and train

In [36]:
X_train, X_test = df[:m_train], df[m_train:]
y_train, _ = target[:m_train], target[m_train:]
n = X_train.shape[1]
X_train.shape, y_train.shape

((1460, 110), (1460,))

In [40]:
df_ohe = pd.get_dummies(df)
df_ohe[:] = StandardScaler().fit_transform(df_ohe)
df_ohe[target_col] = np.log(target)

df_ohe_train = df_ohe[:m_train].copy()

_ = setup(data=df_ohe_train, target='SalePrice')
best = compare_models()

,Description,Value
0,Session id,5038
1,Target,SalePrice
2,Target type,Regression
3,Data shape,"(1460, 288)"
4,Train data shape,"(1021, 288)"
5,Test data shape,"(439, 288)"
6,Numeric features,287
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.0841,0.0158,0.1229,0.9018,0.0095,0.0070,5.0590
gbr,Gradient Boosting Regressor,0.0912,0.0179,0.1316,0.8888,0.0102,0.0076,0.2750
et,Extra Trees Regressor,0.0925,0.0192,0.1372,0.8800,0.0106,0.0077,0.4380
lightgbm,Light Gradient Boosting Machine,0.0954,0.0197,0.1383,0.8765,0.0107,0.0080,0.1030
rf,Random Forest Regressor,0.0947,0.0205,0.1414,0.8720,0.0109,0.0079,0.4530
xgboost,Extreme Gradient Boosting,0.0995,0.0208,0.1420,0.8703,0.0109,0.0083,0.3980
br,Bayesian Ridge,0.0878,0.0229,0.1408,0.8600,0.0107,0.0074,0.0330
ridge,Ridge Regression,0.0940,0.0262,0.1523,0.8386,0.0116,0.0079,0.0270
omp,Orthogonal Matching Pursuit,0.0939,0.0287,0.1573,0.8233,0.0119,0.0079,0.0260
ada,AdaBoost Regressor,0.1261,0.0291,0.1687,0.8187,0.0130,0.0105,0.1630


# Training - CatBoost Grid Search 

In [ ]:
cat_features = df.select_dtypes(object).columns.to_list()
model = CatBoostRegressor(random_state=10, cat_features=cat_features)
kwargs = {
    "n_estimators": (2.5 * n, 3 * n, 4 * n, ),
    "max_depth": (8, 10, 12, ),
    "subsample": (0.65, ),
    "reg_lambda": (0.05, 0.1, 0.15, ), 
}
clf = GridSearchCV(model, kwargs, verbose=1, n_jobs=2)
clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
0:	learn: 77202.4313556	total: 90.8ms	remaining: 24.9s
1:	learn: 75600.3149622	total: 114ms	remaining: 15.6s
2:	learn: 74004.4459315	total: 167ms	remaining: 15.2s
3:	learn: 72423.2616568	total: 212ms	remaining: 14.3s
4:	learn: 70944.1593374	total: 259ms	remaining: 14s
5:	learn: 69439.0117376	total: 323ms	remaining: 14.5s
6:	learn: 67899.7059995	total: 362ms	remaining: 13.9s
7:	learn: 66569.6692585	total: 428ms	remaining: 14.3s
8:	learn: 65050.3079078	total: 508ms	remaining: 15s
9:	learn: 63681.4913309	total: 552ms	remaining: 14.6s
10:	learn: 62377.5356181	total: 598ms	remaining: 14.4s
11:	learn: 61071.4001056	total: 641ms	remaining: 14.1s
12:	learn: 59852.7830788	total: 673ms	remaining: 13.6s
13:	learn: 58674.5716591	total: 727ms	remaining: 13.6s
14:	learn: 57497.4602176	total: 770ms	remaining: 13.4s
15:	learn: 56363.2718149	total: 836ms	remaining: 13.5s
16:	learn: 55199.4028654	total: 886ms	remaining: 13.4s
17:	learn: 54081

0:	learn: 78782.4802860	total: 136ms	remaining: 37.2s
1:	learn: 77023.1908041	total: 217ms	remaining: 29.6s
2:	learn: 75374.3960256	total: 265ms	remaining: 24s
3:	learn: 73750.0944581	total: 312ms	remaining: 21.1s
4:	learn: 72242.2946195	total: 371ms	remaining: 20.1s
5:	learn: 70714.9647432	total: 428ms	remaining: 19.2s
6:	learn: 69318.1174745	total: 484ms	remaining: 18.5s
7:	learn: 67817.9872416	total: 516ms	remaining: 17.2s
8:	learn: 66455.8893186	total: 552ms	remaining: 16.3s
9:	learn: 65066.9812119	total: 618ms	remaining: 16.4s
10:	learn: 63625.5818863	total: 665ms	remaining: 16s
11:	learn: 62256.0236064	total: 744ms	remaining: 16.3s
12:	learn: 60891.0640508	total: 791ms	remaining: 15.9s
13:	learn: 59674.9141203	total: 842ms	remaining: 15.7s
14:	learn: 58530.0001638	total: 896ms	remaining: 15.5s
15:	learn: 57461.8591607	total: 915ms	remaining: 14.8s
16:	learn: 56318.7884863	total: 946ms	remaining: 14.4s
17:	learn: 55246.5344258	total: 986ms	remaining: 14.1s
18:	learn: 54173.3547080

149:	learn: 14494.2748657	total: 7.93s	remaining: 6.61s
150:	learn: 14438.2239695	total: 7.96s	remaining: 6.54s
151:	learn: 14387.4828296	total: 8.05s	remaining: 6.51s
152:	learn: 14324.7280504	total: 8.09s	remaining: 6.45s
153:	learn: 14260.1933355	total: 8.13s	remaining: 6.39s
154:	learn: 14195.2688591	total: 8.18s	remaining: 6.33s
155:	learn: 14147.5491350	total: 8.21s	remaining: 6.26s
156:	learn: 14070.9967810	total: 8.25s	remaining: 6.2s
157:	learn: 14016.4717975	total: 8.29s	remaining: 6.14s
158:	learn: 13978.8155372	total: 8.35s	remaining: 6.09s
159:	learn: 13944.6512824	total: 8.42s	remaining: 6.05s
160:	learn: 13894.6796206	total: 8.48s	remaining: 6s
161:	learn: 13833.0019634	total: 8.56s	remaining: 5.97s
162:	learn: 13789.8643048	total: 8.62s	remaining: 5.92s
163:	learn: 13749.8046890	total: 8.69s	remaining: 5.88s
164:	learn: 13709.6903308	total: 8.75s	remaining: 5.83s
165:	learn: 13663.3293011	total: 8.8s	remaining: 5.78s
166:	learn: 13576.5108531	total: 8.88s	remaining: 5.7

21:	learn: 51028.9045353	total: 1.09s	remaining: 12.5s
22:	learn: 50028.5557302	total: 1.14s	remaining: 12.5s
23:	learn: 49152.7785512	total: 1.23s	remaining: 12.8s
24:	learn: 48336.7974078	total: 1.26s	remaining: 12.6s
25:	learn: 47489.4122110	total: 1.3s	remaining: 12.4s
26:	learn: 46685.5748647	total: 1.36s	remaining: 12.5s
27:	learn: 45846.1077798	total: 1.41s	remaining: 12.4s
28:	learn: 45184.3163771	total: 1.48s	remaining: 12.5s
29:	learn: 44367.7409492	total: 1.54s	remaining: 12.6s
30:	learn: 43525.1079028	total: 1.59s	remaining: 12.5s
31:	learn: 42816.8252498	total: 1.64s	remaining: 12.5s
32:	learn: 42117.4724135	total: 1.68s	remaining: 12.3s
33:	learn: 41488.9236967	total: 1.73s	remaining: 12.2s
34:	learn: 40787.4100109	total: 1.78s	remaining: 12.2s
35:	learn: 40191.5869876	total: 1.83s	remaining: 12.2s
36:	learn: 39575.9751016	total: 1.89s	remaining: 12.2s
37:	learn: 38895.9043623	total: 1.95s	remaining: 12.2s
38:	learn: 38365.3493440	total: 1.99s	remaining: 12s
39:	learn: 37

41:	learn: 35943.4095994	total: 1.79s	remaining: 9.94s
42:	learn: 35306.6909381	total: 1.85s	remaining: 9.99s
43:	learn: 34717.4200757	total: 1.87s	remaining: 9.84s
44:	learn: 34211.5776628	total: 1.9s	remaining: 9.71s
45:	learn: 33757.6384703	total: 1.94s	remaining: 9.66s
46:	learn: 33232.8702813	total: 2.03s	remaining: 9.84s
47:	learn: 32781.6086409	total: 2.09s	remaining: 9.89s
48:	learn: 32306.3217972	total: 2.13s	remaining: 9.83s
49:	learn: 31804.4256181	total: 2.2s	remaining: 9.9s
50:	learn: 31358.3250317	total: 2.25s	remaining: 9.9s
51:	learn: 30867.3942193	total: 2.31s	remaining: 9.93s
52:	learn: 30464.9443287	total: 2.35s	remaining: 9.84s
53:	learn: 30057.6222989	total: 2.41s	remaining: 9.86s
54:	learn: 29622.3274938	total: 2.45s	remaining: 9.8s
55:	learn: 29166.2060209	total: 2.48s	remaining: 9.71s
56:	learn: 28736.0332471	total: 2.53s	remaining: 9.68s
57:	learn: 28352.3326797	total: 2.6s	remaining: 9.73s
58:	learn: 28024.6684285	total: 2.65s	remaining: 9.7s
59:	learn: 27674.

...

139:	learn: 15071.0087049	total: 3.58s	remaining: 4.86s
140:	learn: 15010.4043993	total: 3.61s	remaining: 4.84s
141:	learn: 14966.9879792	total: 3.63s	remaining: 4.81s
142:	learn: 14894.2090251	total: 3.66s	remaining: 4.79s
143:	learn: 14839.3321735	total: 3.69s	remaining: 4.76s
144:	learn: 14768.2235194	total: 3.72s	remaining: 4.75s
145:	learn: 14730.2619609	total: 3.74s	remaining: 4.72s
146:	learn: 14681.9012202	total: 3.77s	remaining: 4.69s
147:	learn: 14629.1597566	total: 3.8s	remaining: 4.68s
148:	learn: 14582.6096901	total: 3.83s	remaining: 4.65s
149:	learn: 14494.2748657	total: 3.86s	remaining: 4.63s
150:	learn: 14438.2239695	total: 3.88s	remaining: 4.6s
151:	learn: 14387.4828296	total: 3.91s	remaining: 4.58s
152:	learn: 14324.7280504	total: 3.94s	remaining: 4.55s
153:	learn: 14260.1933355	total: 3.96s	remaining: 4.53s
154:	learn: 14195.2688591	total: 3.99s	remaining: 4.5s
155:	learn: 14147.5491350	total: 4.01s	remaining: 4.47s
156:	learn: 14070.9967810	total: 4.04s	remaining: 4

286:	learn: 9662.2188434	total: 8.21s	remaining: 1.23s
287:	learn: 9645.6164397	total: 8.25s	remaining: 1.2s
288:	learn: 9613.4323849	total: 8.28s	remaining: 1.17s
289:	learn: 9591.0391014	total: 8.31s	remaining: 1.15s
290:	learn: 9570.5777840	total: 8.34s	remaining: 1.12s
291:	learn: 9521.0592765	total: 8.37s	remaining: 1.09s
292:	learn: 9505.7124454	total: 8.4s	remaining: 1.06s
293:	learn: 9482.3958649	total: 8.44s	remaining: 1.03s
294:	learn: 9450.8555517	total: 8.47s	remaining: 1s
295:	learn: 9407.3834660	total: 8.5s	remaining: 976ms
296:	learn: 9384.5023445	total: 8.53s	remaining: 948ms
297:	learn: 9354.2617203	total: 8.56s	remaining: 919ms
298:	learn: 9341.1391384	total: 8.59s	remaining: 890ms
299:	learn: 9323.9004512	total: 8.61s	remaining: 861ms
300:	learn: 9296.0103518	total: 8.64s	remaining: 832ms
301:	learn: 9272.4585747	total: 8.67s	remaining: 804ms
302:	learn: 9257.7493232	total: 8.7s	remaining: 775ms
303:	learn: 9228.0843626	total: 8.73s	remaining: 747ms
304:	learn: 9204.

105:	learn: 17329.4807666	total: 2.88s	remaining: 6.08s
106:	learn: 17214.1928205	total: 2.91s	remaining: 6.06s
107:	learn: 17089.9084035	total: 2.93s	remaining: 6.03s
108:	learn: 16983.1417008	total: 2.96s	remaining: 6.01s
109:	learn: 16863.3169479	total: 2.99s	remaining: 5.99s
110:	learn: 16762.4075255	total: 3.02s	remaining: 5.97s
111:	learn: 16655.9159398	total: 3.05s	remaining: 5.94s
112:	learn: 16531.0915586	total: 3.08s	remaining: 5.91s
113:	learn: 16436.8748578	total: 3.1s	remaining: 5.87s
114:	learn: 16367.2786692	total: 3.12s	remaining: 5.84s
115:	learn: 16262.3338102	total: 3.15s	remaining: 5.81s
116:	learn: 16152.1223894	total: 3.17s	remaining: 5.77s
117:	learn: 16043.7896775	total: 3.21s	remaining: 5.76s
118:	learn: 15974.7203753	total: 3.23s	remaining: 5.73s
119:	learn: 15866.8119407	total: 3.26s	remaining: 5.71s
120:	learn: 15800.9835058	total: 3.29s	remaining: 5.69s
121:	learn: 15714.9939365	total: 3.32s	remaining: 5.66s
122:	learn: 15642.4353576	total: 3.35s	remaining:

251:	learn: 10277.7926172	total: 7.27s	remaining: 2.25s
252:	learn: 10266.8798811	total: 7.3s	remaining: 2.22s
253:	learn: 10244.8463568	total: 7.33s	remaining: 2.19s
254:	learn: 10226.1334551	total: 7.36s	remaining: 2.16s
255:	learn: 10198.6983126	total: 7.39s	remaining: 2.13s
256:	learn: 10157.0192059	total: 7.42s	remaining: 2.11s
257:	learn: 10128.9802537	total: 7.44s	remaining: 2.08s
258:	learn: 10117.8574055	total: 7.48s	remaining: 2.05s
259:	learn: 10069.1698743	total: 7.51s	remaining: 2.02s
260:	learn: 10042.8452599	total: 7.55s	remaining: 2s
261:	learn: 10011.4173552	total: 7.58s	remaining: 1.97s
262:	learn: 9989.6778140	total: 7.61s	remaining: 1.94s
263:	learn: 9972.7397508	total: 7.64s	remaining: 1.91s
264:	learn: 9920.3515237	total: 7.66s	remaining: 1.88s
265:	learn: 9899.9688912	total: 7.69s	remaining: 1.85s
266:	learn: 9889.8739108	total: 7.72s	remaining: 1.82s
267:	learn: 9857.9510178	total: 7.75s	remaining: 1.79s
268:	learn: 9824.4739036	total: 7.77s	remaining: 1.76s
269

127:	learn: 16505.3521953	total: 3.72s	remaining: 5.87s
128:	learn: 16451.7769789	total: 3.74s	remaining: 5.83s
129:	learn: 16380.4913587	total: 3.77s	remaining: 5.81s
130:	learn: 16319.7365308	total: 3.8s	remaining: 5.77s
131:	learn: 16257.6213181	total: 3.83s	remaining: 5.74s
132:	learn: 16177.4815641	total: 3.85s	remaining: 5.71s
133:	learn: 16103.4641002	total: 3.88s	remaining: 5.68s
134:	learn: 16048.3100435	total: 3.92s	remaining: 5.66s
135:	learn: 16009.2874536	total: 3.94s	remaining: 5.63s
136:	learn: 15952.9269672	total: 3.97s	remaining: 5.6s
137:	learn: 15894.9683989	total: 4s	remaining: 5.57s
138:	learn: 15831.7288857	total: 4.04s	remaining: 5.54s
139:	learn: 15714.3672526	total: 4.06s	remaining: 5.51s
140:	learn: 15675.9909270	total: 4.1s	remaining: 5.5s
141:	learn: 15622.5665639	total: 4.13s	remaining: 5.47s
142:	learn: 15585.8107082	total: 4.16s	remaining: 5.44s
143:	learn: 15513.8470391	total: 4.19s	remaining: 5.41s
144:	learn: 15454.9689080	total: 4.21s	remaining: 5.38s

# Training - CatBoost with Best Parameters

In [37]:
cat_cols = df.select_dtypes(object).columns.to_list()
model = CatBoostRegressor(max_depth=8, random_seed=10,
                          subsample=0.65, n_estimators=1000,
                          cat_features=cat_cols)
model.fit(X_train, y_train)

Learning rate set to 0.043466
0:	learn: 77189.7092001	total: 60.4ms	remaining: 1m
1:	learn: 75176.6507089	total: 99ms	remaining: 49.4s
2:	learn: 73015.2109472	total: 166ms	remaining: 55s
3:	learn: 70993.0638332	total: 205ms	remaining: 50.9s
4:	learn: 69133.7059883	total: 240ms	remaining: 47.8s
5:	learn: 67377.9822727	total: 270ms	remaining: 44.8s
6:	learn: 65548.5983766	total: 310ms	remaining: 44s
7:	learn: 63759.1781795	total: 389ms	remaining: 48.3s
8:	learn: 62114.0645366	total: 414ms	remaining: 45.5s
9:	learn: 60542.8394749	total: 475ms	remaining: 47s
10:	learn: 59088.6604518	total: 524ms	remaining: 47.1s
11:	learn: 57692.9417415	total: 580ms	remaining: 47.7s
12:	learn: 56400.0811135	total: 624ms	remaining: 47.4s
13:	learn: 55090.9916362	total: 663ms	remaining: 46.7s
14:	learn: 53866.1642996	total: 711ms	remaining: 46.7s
15:	learn: 52842.2339597	total: 801ms	remaining: 49.3s
16:	learn: 51657.2958549	total: 875ms	remaining: 50.6s
17:	learn: 50525.2614529	total: 913ms	remaining: 49.8s

153:	learn: 16251.2099743	total: 7.98s	remaining: 43.8s
154:	learn: 16215.1567306	total: 8.03s	remaining: 43.8s
155:	learn: 16180.1954889	total: 8.11s	remaining: 43.9s
156:	learn: 16136.9889314	total: 8.18s	remaining: 43.9s
157:	learn: 16100.4628446	total: 8.24s	remaining: 43.9s
158:	learn: 16059.6892414	total: 8.3s	remaining: 43.9s
159:	learn: 16001.0122897	total: 8.36s	remaining: 43.9s
160:	learn: 15995.2700753	total: 8.46s	remaining: 44.1s
161:	learn: 15951.2798522	total: 8.49s	remaining: 43.9s
162:	learn: 15918.2619635	total: 8.53s	remaining: 43.8s
163:	learn: 15906.7387309	total: 8.57s	remaining: 43.7s
164:	learn: 15871.2217769	total: 8.64s	remaining: 43.8s
165:	learn: 15811.4876730	total: 8.7s	remaining: 43.7s
166:	learn: 15767.0892318	total: 8.74s	remaining: 43.6s
167:	learn: 15698.3794626	total: 8.82s	remaining: 43.7s
168:	learn: 15662.9527473	total: 8.88s	remaining: 43.7s
169:	learn: 15603.2971600	total: 8.93s	remaining: 43.6s
170:	learn: 15564.8597401	total: 8.98s	remaining: 

303:	learn: 11668.0622846	total: 16.6s	remaining: 38.1s
304:	learn: 11638.3555046	total: 16.7s	remaining: 38.1s
305:	learn: 11592.3029881	total: 16.8s	remaining: 38s
306:	learn: 11565.8935623	total: 16.8s	remaining: 38s
307:	learn: 11560.9771156	total: 16.9s	remaining: 37.9s
308:	learn: 11549.3797916	total: 16.9s	remaining: 37.9s
309:	learn: 11532.9066217	total: 17s	remaining: 37.9s
310:	learn: 11527.2387756	total: 17s	remaining: 37.8s
311:	learn: 11513.7483912	total: 17.1s	remaining: 37.8s
312:	learn: 11491.6004097	total: 17.2s	remaining: 37.7s
313:	learn: 11463.1080260	total: 17.2s	remaining: 37.7s
314:	learn: 11435.8235236	total: 17.3s	remaining: 37.6s
315:	learn: 11422.8150995	total: 17.4s	remaining: 37.6s
316:	learn: 11394.7423668	total: 17.4s	remaining: 37.5s
317:	learn: 11382.4521649	total: 17.4s	remaining: 37.4s
318:	learn: 11380.2376181	total: 17.5s	remaining: 37.4s
319:	learn: 11365.9144736	total: 17.5s	remaining: 37.3s
320:	learn: 11333.3949517	total: 17.6s	remaining: 37.2s


455:	learn: 9042.6370817	total: 24.9s	remaining: 29.7s
456:	learn: 9024.8672691	total: 25s	remaining: 29.7s
457:	learn: 9010.0533892	total: 25s	remaining: 29.6s
458:	learn: 9005.6791863	total: 25.1s	remaining: 29.6s
459:	learn: 8991.3469295	total: 25.2s	remaining: 29.6s
460:	learn: 8979.3432462	total: 25.3s	remaining: 29.5s
461:	learn: 8961.2833047	total: 25.3s	remaining: 29.5s
462:	learn: 8949.1304097	total: 25.4s	remaining: 29.4s
463:	learn: 8933.2820562	total: 25.4s	remaining: 29.4s
464:	learn: 8922.4461966	total: 25.5s	remaining: 29.3s
465:	learn: 8915.4297769	total: 25.5s	remaining: 29.3s
466:	learn: 8900.0734861	total: 25.6s	remaining: 29.2s
467:	learn: 8882.6153660	total: 25.6s	remaining: 29.1s
468:	learn: 8872.5683087	total: 25.7s	remaining: 29.1s
469:	learn: 8865.4752525	total: 25.8s	remaining: 29.1s
470:	learn: 8853.7376511	total: 25.8s	remaining: 29s
471:	learn: 8842.4835500	total: 25.9s	remaining: 29s
472:	learn: 8833.8141323	total: 25.9s	remaining: 28.9s
473:	learn: 8808.2

609:	learn: 7209.9800567	total: 33.4s	remaining: 21.4s
610:	learn: 7196.6487709	total: 33.5s	remaining: 21.3s
611:	learn: 7177.6474836	total: 33.5s	remaining: 21.3s
612:	learn: 7163.5853734	total: 33.6s	remaining: 21.2s
613:	learn: 7151.1359981	total: 33.6s	remaining: 21.1s
614:	learn: 7128.5905092	total: 33.6s	remaining: 21.1s
615:	learn: 7116.7909731	total: 33.7s	remaining: 21s
616:	learn: 7102.1812980	total: 33.7s	remaining: 20.9s
617:	learn: 7089.7251067	total: 33.8s	remaining: 20.9s
618:	learn: 7075.7560962	total: 33.9s	remaining: 20.9s
619:	learn: 7068.6336554	total: 33.9s	remaining: 20.8s
620:	learn: 7054.6847045	total: 34s	remaining: 20.7s
621:	learn: 7042.4347306	total: 34s	remaining: 20.7s
622:	learn: 7032.6136921	total: 34.1s	remaining: 20.6s
623:	learn: 7023.3767215	total: 34.1s	remaining: 20.6s
624:	learn: 7004.8473925	total: 34.2s	remaining: 20.5s
625:	learn: 6985.8376082	total: 34.2s	remaining: 20.4s
626:	learn: 6970.7426801	total: 34.2s	remaining: 20.4s
627:	learn: 6962

759:	learn: 5934.4955525	total: 41.8s	remaining: 13.2s
760:	learn: 5920.7521596	total: 41.9s	remaining: 13.2s
761:	learn: 5915.1793760	total: 41.9s	remaining: 13.1s
762:	learn: 5904.1820604	total: 42s	remaining: 13s
763:	learn: 5890.2247654	total: 42s	remaining: 13s
764:	learn: 5878.1432412	total: 42.1s	remaining: 12.9s
765:	learn: 5874.3163506	total: 42.1s	remaining: 12.9s
766:	learn: 5868.0768089	total: 42.2s	remaining: 12.8s
767:	learn: 5855.7668242	total: 42.2s	remaining: 12.7s
768:	learn: 5849.2404314	total: 42.2s	remaining: 12.7s
769:	learn: 5840.5950517	total: 42.3s	remaining: 12.6s
770:	learn: 5831.0369359	total: 42.4s	remaining: 12.6s
771:	learn: 5825.5759574	total: 42.4s	remaining: 12.5s
772:	learn: 5821.8244610	total: 42.5s	remaining: 12.5s
773:	learn: 5809.6179974	total: 42.5s	remaining: 12.4s
774:	learn: 5802.5285423	total: 42.6s	remaining: 12.4s
775:	learn: 5801.5319258	total: 42.6s	remaining: 12.3s
776:	learn: 5795.7700666	total: 42.7s	remaining: 12.3s
777:	learn: 5778.2

914:	learn: 4874.0025333	total: 50.4s	remaining: 4.68s
915:	learn: 4865.5302376	total: 50.5s	remaining: 4.63s
916:	learn: 4861.8264549	total: 50.6s	remaining: 4.58s
917:	learn: 4858.0989504	total: 50.6s	remaining: 4.52s
918:	learn: 4843.4182524	total: 50.7s	remaining: 4.47s
919:	learn: 4839.7032331	total: 50.8s	remaining: 4.42s
920:	learn: 4838.4408502	total: 50.9s	remaining: 4.36s
921:	learn: 4834.9149421	total: 50.9s	remaining: 4.31s
922:	learn: 4826.0241007	total: 51s	remaining: 4.25s
923:	learn: 4822.5340614	total: 51s	remaining: 4.19s
924:	learn: 4812.4712841	total: 51s	remaining: 4.14s
925:	learn: 4806.2363557	total: 51.1s	remaining: 4.08s
926:	learn: 4805.7568911	total: 51.1s	remaining: 4.03s
927:	learn: 4803.3916493	total: 51.2s	remaining: 3.97s
928:	learn: 4793.2228504	total: 51.2s	remaining: 3.92s
929:	learn: 4790.9246342	total: 51.3s	remaining: 3.86s
930:	learn: 4788.4040874	total: 51.3s	remaining: 3.8s
931:	learn: 4787.9466789	total: 51.4s	remaining: 3.75s
932:	learn: 4784.

In [49]:
fe = model.get_feature_importance(prettified=True)
fe.head(20)

,Feature Id,Importances
0,OverallQual,10.269672
1,GrLivArea,8.350941
2,weighted_home_area,7.304981
3,Neighborhood,4.582893
4,weighted_bsmt_area,3.632355
5,BsmtFinSF1,3.177597
6,ExterQual,2.726940
7,total_area_hq,2.609414
8,area_qu,2.599534
9,YearBuilt,2.560371


In [46]:
sub_name = "data/catboost0.csv"
pd.DataFrame(model.predict(X_test), 
            index=range(1461, len(df)+1), 
            columns=['SalePrice']).reset_index().\
            rename(columns={'index': 'id'}).to_csv(sub_name, index=False)

In [50]:
model.best_score_

{'learn': {'RMSE': 4426.4999229219475}}